# Agentic Learning Platform - Database Analysis

This notebook provides tools for exploring and analyzing student performance data from the learning platform.

## Database Schema

- **students**: Student profiles (name, grade_level)
- **sessions**: Learning sessions (start_time, end_time, module_id)
- **activity_attempts**: Quiz attempts (score, total, difficulty, item_results)
- **chat_messages**: Conversation history between student and agent

## Setup

In [2]:
# Import libraries
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime
from pathlib import Path

# Set up plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Database path
DB_PATH = 'learning.db'

print(f"Database: {DB_PATH}")
print(f"Exists: {Path(DB_PATH).exists()}")

Database: learning.db
Exists: True


## Database Connection

In [3]:
# Connect to database
conn = sqlite3.connect(DB_PATH)

# Helper function to run queries
def query(sql, params=None):
    """Execute SQL query and return DataFrame"""
    if params:
        return pd.read_sql_query(sql, conn, params=params)
    return pd.read_sql_query(sql, conn)

print("✓ Connected to database")

✓ Connected to database


## 1. Basic Data Exploration

### View All Tables

In [4]:
# List all tables
tables = query("""
    SELECT name FROM sqlite_master 
    WHERE type='table'
    ORDER BY name
""")

print("Tables in database:")
display(tables)

Tables in database:


,name
0,activity_attempts
1,chat_messages
2,sessions
3,students


In [19]:
act = query("""
    SELECT item_results from activity_attempts
""")

for index, row in act.iterrows():
    print(row[0])

"[{\"problem_id\": 1, \"expression\": \"3 \\u00d7 4\", \"correct_answer\": 12, \"student_answer\": 12, \"correct\": true, \"used_retry\": false}, {\"problem_id\": 2, \"expression\": \"7 \\u00d7 8\", \"correct_answer\": 56, \"student_answer\": 56, \"correct\": true, \"used_retry\": true}, {\"problem_id\": 3, \"expression\": \"6 \\u00d7 12\", \"correct_answer\": 72, \"student_answer\": 72, \"correct\": true, \"used_retry\": false}, {\"problem_id\": 4, \"expression\": \"9 \\u00d7 5\", \"correct_answer\": 45, \"student_answer\": 45, \"correct\": true, \"used_retry\": false}, {\"problem_id\": 5, \"expression\": \"8 \\u00d7 3\", \"correct_answer\": 24, \"student_answer\": 24, \"correct\": true, \"used_retry\": true}]"
"[{\"problem_id\": 1, \"expression\": \"3 \\u00d7 4\", \"correct_answer\": 12, \"student_answer\": 12, \"correct\": true, \"used_retry\": false}, {\"problem_id\": 2, \"expression\": \"7 \\u00d7 8\", \"correct_answer\": 56, \"student_answer\": 56, \"correct\": true, \"used_retry

/var/folders/vt/q4zq5m9x3zs7dl6ysjlz9lsc0000gn/T/ipykernel_89977/2744663576.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[0])


### Students

In [ ]:
# View all students
students = query("""
    SELECT 
        student_id,
        name,
        grade_level,
        created_at,
        COUNT(DISTINCT s.session_id) as total_sessions,
        COUNT(DISTINCT a.attempt_id) as total_attempts
    FROM students st
    LEFT JOIN sessions s ON st.student_id = s.student_id
    LEFT JOIN activity_attempts a ON st.student_id = a.student_id
    GROUP BY st.student_id
    ORDER BY created_at DESC
""")

print(f"Total Students: {len(students)}")
display(students)

### Sessions

In [ ]:
# View all sessions with student names
sessions = query("""
    SELECT 
        s.session_id,
        st.name as student_name,
        s.module_id,
        s.start_time,
        s.end_time,
        COUNT(a.attempt_id) as num_attempts
    FROM sessions s
    JOIN students st ON s.student_id = st.student_id
    LEFT JOIN activity_attempts a ON s.session_id = a.session_id
    GROUP BY s.session_id
    ORDER BY s.start_time DESC
""")

print(f"Total Sessions: {len(sessions)}")
display(sessions.head(10))

### Activity Attempts

In [ ]:
# View all activity attempts
attempts = query("""
    SELECT 
        a.attempt_id,
        st.name as student_name,
        a.module,
        a.activity,
        a.score,
        a.total,
        ROUND(a.score * 100.0 / a.total, 1) as percentage,
        a.difficulty,
        a.date
    FROM activity_attempts a
    JOIN students st ON a.student_id = st.student_id
    ORDER BY a.date DESC
""")

print(f"Total Attempts: {len(attempts)}")
display(attempts.head(10))

## 2. Performance Analytics

### Overall Statistics

In [ ]:
# Calculate overall statistics
stats = query("""
    SELECT 
        COUNT(DISTINCT student_id) as total_students,
        COUNT(DISTINCT session_id) as total_sessions,
        COUNT(*) as total_attempts,
        ROUND(AVG(score * 100.0 / total), 1) as avg_percentage,
        ROUND(MIN(score * 100.0 / total), 1) as min_percentage,
        ROUND(MAX(score * 100.0 / total), 1) as max_percentage
    FROM activity_attempts
""")

print("\n📊 Overall Statistics")
print("=" * 50)
display(stats)

### Performance by Student

In [ ]:
# Performance by student
student_performance = query("""
    SELECT 
        st.name as student_name,
        COUNT(*) as num_attempts,
        ROUND(AVG(a.score * 100.0 / a.total), 1) as avg_percentage,
        SUM(a.score) as total_correct,
        SUM(a.total) as total_questions,
        MIN(a.date) as first_attempt,
        MAX(a.date) as last_attempt
    FROM activity_attempts a
    JOIN students st ON a.student_id = st.student_id
    GROUP BY st.student_id
    ORDER BY avg_percentage DESC
""")

print("\n👥 Performance by Student")
print("=" * 50)
display(student_performance)

### Performance by Module

In [ ]:
# Performance by module
module_performance = query("""
    SELECT 
        module,
        activity,
        COUNT(*) as num_attempts,
        ROUND(AVG(score * 100.0 / total), 1) as avg_percentage,
        COUNT(DISTINCT student_id) as num_students
    FROM activity_attempts
    GROUP BY module, activity
    ORDER BY module, activity
""")

print("\n📚 Performance by Module")
print("=" * 50)
display(module_performance)

## 3. Detailed Item Analysis

### Extract Item-Level Results

In [ ]:
# Get all attempts with item results
attempts_with_items = query("""
    SELECT 
        a.attempt_id,
        st.name as student_name,
        a.module,
        a.item_results,
        a.date
    FROM activity_attempts a
    JOIN students st ON a.student_id = st.student_id
    ORDER BY a.date DESC
""")

# Parse JSON item results
item_data = []
for _, row in attempts_with_items.iterrows():
    try:
        items = json.loads(row['item_results'])
        for item in items:
            item_data.append({
                'student_name': row['student_name'],
                'module': row['module'],
                'problem_id': item.get('problem_id'),
                'expression': item.get('expression'),
                'correct_answer': item.get('correct_answer'),
                'student_answer': item.get('student_answer'),
                'correct': item.get('correct'),
                'used_retry': item.get('used_retry', False),
                'date': row['date']
            })
    except:
        pass

items_df = pd.DataFrame(item_data)

if len(items_df) > 0:
    print(f"\n📝 Item-Level Results: {len(items_df)} problems attempted")
    display(items_df.head(10))
else:
    print("No item-level data available yet. Run some quizzes first!")

### Problem Difficulty Analysis

In [ ]:
# Analyze which problems are hardest
if len(items_df) > 0:
    problem_stats = items_df.groupby('expression').agg({
        'correct': ['count', 'sum', 'mean'],
        'used_retry': 'sum'
    }).round(3)
    
    problem_stats.columns = ['attempts', 'correct', 'success_rate', 'retries_used']
    problem_stats = problem_stats.sort_values('success_rate')
    
    print("\n🎯 Problem Difficulty (sorted by success rate)")
    print("=" * 50)
    display(problem_stats)
else:
    print("No problem data available yet.")

## 4. Visualizations

### Score Distribution

In [ ]:
if len(attempts) > 0:
    plt.figure(figsize=(10, 6))
    plt.hist(attempts['percentage'], bins=20, edgecolor='black', alpha=0.7)
    plt.xlabel('Score (%)')
    plt.ylabel('Number of Attempts')
    plt.title('Distribution of Quiz Scores')
    plt.axvline(attempts['percentage'].mean(), color='red', linestyle='--', 
                label=f'Mean: {attempts["percentage"].mean():.1f}%')
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.show()
else:
    print("No data to visualize yet.")

### Performance Over Time

In [ ]:
if len(attempts) > 0:
    # Convert date to datetime
    attempts['date'] = pd.to_datetime(attempts['date'])
    
    # Plot performance over time for each student
    plt.figure(figsize=(12, 6))
    for student in attempts['student_name'].unique():
        student_data = attempts[attempts['student_name'] == student].sort_values('date')
        plt.plot(student_data['date'], student_data['percentage'], 
                marker='o', label=student, linewidth=2)
    
    plt.xlabel('Date')
    plt.ylabel('Score (%)')
    plt.title('Student Performance Over Time')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No data to visualize yet.")

### Problem Success Rates

In [ ]:
if len(items_df) > 0:
    # Calculate success rate by problem
    problem_success = items_df.groupby('expression')['correct'].agg(['sum', 'count'])
    problem_success['success_rate'] = problem_success['sum'] / problem_success['count'] * 100
    problem_success = problem_success.sort_values('success_rate')
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(problem_success)), problem_success['success_rate'])
    plt.yticks(range(len(problem_success)), problem_success.index)
    plt.xlabel('Success Rate (%)')
    plt.title('Success Rate by Problem')
    plt.axvline(50, color='red', linestyle='--', alpha=0.5, label='50%')
    plt.legend()
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No problem data available yet.")

## 5. Chat Message Analysis

In [ ]:
# View chat messages
messages = query("""
    SELECT 
        m.message_id,
        st.name as student_name,
        m.agent_type,
        m.sender,
        m.message,
        m.timestamp
    FROM chat_messages m
    JOIN sessions s ON m.session_id = s.session_id
    JOIN students st ON s.student_id = st.student_id
    ORDER BY m.timestamp DESC
    LIMIT 20
""")

if len(messages) > 0:
    print(f"\n💬 Recent Chat Messages: {len(messages)}")
    display(messages)
else:
    print("No chat messages recorded yet.")

## 6. Custom Queries

### Query Template

Use this cell to run your own custom SQL queries:

In [ ]:
# Custom query - modify as needed
custom_query = """
    SELECT * FROM students
    LIMIT 10
"""

result = query(custom_query)
display(result)

## 7. Export Data

In [ ]:
# Export data to CSV files
def export_data():
    """Export all tables to CSV files"""
    export_dir = Path('exports')
    export_dir.mkdir(exist_ok=True)
    
    # Export each table
    tables_to_export = {
        'students': 'SELECT * FROM students',
        'sessions': 'SELECT * FROM sessions',
        'activity_attempts': 'SELECT * FROM activity_attempts',
        'chat_messages': 'SELECT * FROM chat_messages'
    }
    
    for table_name, sql in tables_to_export.items():
        df = query(sql)
        filepath = export_dir / f'{table_name}.csv'
        df.to_csv(filepath, index=False)
        print(f"✓ Exported {table_name} to {filepath}")
    
    print(f"\n✓ All data exported to {export_dir}/")

# Uncomment to export:
# export_data()

## Cleanup

In [ ]:
# Close database connection
conn.close()
print("✓ Database connection closed")